### **Atividade Complementar de Bioinformática**
#### André Nakano Miyake - Caso 2

Comecei configurando o ambiente, instalando todas as bibliotecas necessárias num ambiente WSL Ubuntu. 
> Descrição: Paciente lactente bebê, sexo masculino, 6 meses, apresentada na consulta de pediatria geneticista devido a triagem neonatal alterada evidenciando níveis elevados de fenilalanina e atraso incipiente no desenvolvimento neuropsicomotor.

> Procedimento: Painel genético específico para análise de variantes no gene PAH, realizado a partir de amostra de sangue.

> Objetivo: Identificar variantes genéticas responsáveis pela fenilcetonúria (PKU), fornecendo base para diagnóstico precoce e orientações terapêuticas individualizadas para prevenção de complicações neurológicas.


#### 1. Controle de qualidade

In [ ]:
%%bash
# Gerando o relatório da qualidade dos reads de sequenciamento
fastqc -o ../fastqc/ ../raw/caso2-sindrome-metabolica_R1.fastq.gz ../raw/caso2-sindrome-metabolica_R2.fastq.gz

In [ ]:
%%bash
# Filtrar sequências muito pequenas (-m 90) e cortar as pontas
cutadapt \
  -u 5 -U 5 \
  -u -5 -U -5 \
  -m 90 \
  -o ../cutadapt/caso2-sindrome-metabolica-trimmed_R1.fastq.gz \
  -p ../cutadapt/caso2-sindrome-metabolica-trimmed_R2.fastq.gz \
  ../raw/caso2-sindrome-metabolica_R1.fastq.gz \
  ../raw/caso2-sindrome-metabolica_R2.fastq.gz

In [ ]:
%%bash
# Gerando o relatório da qualidade dos reads de sequenciamento, novamente, após o corte
fastqc -o ../fastqc/ ../cutadapt/caso2-sindrome-metabolica-trimmed_R1.fastq.gz ../cutadapt/caso2-sindrome-metabolica-trimmed_R2.fastq.gz

Ao analisar os HTMLs, notei que a qualidade ainda estava ruim, mas decidi prosseguir com os arquivos que tenho para seguir com o fluxo da atividade. A primeira imagem da esquerda é o original sem cutadapt, e a segunda da direita é a após o corte do cutadapt.

<img src="download.png" width="500" alt="Not trimmed"> <img src="download-1.png" width="500" alt="Trimmed"> 


#### 2. Mapeamento com a referência do genoma humano

In [20]:
%%bash
# Indexando o genoma de referência (Chr12) para o BWA e Samtools
samtools faidx ../reference/Chr12-reference.fasta
bwa index ../reference/Chr12-reference.fasta

[bwa_index] Pack FASTA... 1.18 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=266550618, availableWord=30755168
[BWTIncConstructFromPacked] 10 iterations done. 50732074 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 93723498 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 131930362 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 165884698 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 196059354 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 222874650 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 246704106 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 266550618 characters processed.
[bwt_gen] Finished constructing BWT in 80 iterations.
[bwa_index] 69.95 seconds elapse.
[bwa_index] Update BWT... 0.62 sec
[bwa_index] Pack forward-only FASTA... 0.72 sec
[bwa_index] Construct SA from BWT and Occ..

In [ ]:
%%bash
# O 'bwa mem' lê os arquivos .gz e a referência, e 'samtools sort' recebe a saída de arquivo sam e cria o arquivo .bam
bwa mem ../reference/Chr12-reference.fasta \
  -R '@RG\tID:caso2\tSM:caso2\tPL:ILLUMINA\tLB:lib1\tPU:unit1' \
  ../cutadapt/caso2-sindrome-metabolica-trimmed_R1.fastq.gz \
  ../cutadapt/caso2-sindrome-metabolica-trimmed_R2.fastq.gz \
  | samtools sort -o ../bwa/caso2-sindrome-metabolica-mapped-host.bam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 71422 sequences (10000189 bp)...
[M::process] read 71440 sequences (10000217 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 34793, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (319, 339, 359)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (239, 439)
[M::mem_pestat] mean and std.dev: (340.01, 28.20)
[M::mem_pestat] low and high boundaries for proper pairs: (199, 479)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 71422 reads in 17.108 CPU sec, 17.321 real sec
[M::process] read 71420 sequences (10000023 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 34716, 0, 0)
[M::mem_pestat] skip orientatio

In [ ]:
%%bash
# Cria um arquivo .bai na mesma pasta do BAM (../bwa/)
samtools index ../bwa/caso2-sindrome-metabolica-mapped-host.bam

Abrindo o arquivos no IGV...

#### 3. Chamada de variantes